# LLM Fine-tuning with XTuner

## 1. Download [`XTuner`](https://xtuner.readthedocs.io/zh-cn/latest/get_started/quickstart.html#lora)
```bash
cd /root/autodl-tmp
git clone https://github.com/InternLM/xtuner.git
```

## 2. Setup Runtime Environment
```bash
mkdir -p /root/autodl-tmp/xtuner-env
conda create -p /root/autodl-tmp/xtuner-env python=3.10 -y
conda activate /root/autodl-tmp/xtuner-env

pip install -e '.[all]'
```

## 3. Download pre-trained model
- Create `/root/autodl-tmp/download-model.py`
```bash
vi /root/autodl-tmp/download-model.py
```
- Paste below code
```python
# !pip install modelscope
from modelscope import snapshot_download

snapshot_download('Shanghai_AI_Laboratory/internlm2-chat-1_8b', cache_dir='/root/autodl-tmp/models')
```
- Execute script
```bash
mkdir -p /root/autodl-tmp/models
python /root/autodl-tmp/download-model.py
```

## 4. Download dataset
```bash
mkdir -p /root/autodl-tmp/datasets
cd /root/autodl-tmp/datasets
# Download dataset
wget https://hf-mirror.com/datasets/LooksJuicy/ruozhiba/blob/main/ruozhiba_qa.json
```

## 5. Configure XTuner

- Copy `xtuner/xtuner/configs/internlm/internlm2_chat_1_8b/internlm2_chat_1_8b_qlora_alpaca_e3.py`
```bash
cp /root/autodl-tmp/xtuner/xtuner/configs/internlm/internlm2_chat_1_8b/internlm2_chat_1_8b_qlora_alpaca_e3.py /root/autodl-tmp/internlm2_chat_1_8b_qlora_alpaca_e3_finetune.py
```

- Edit `/root/autodl-tmp/internlm2_chat_1_8b_qlora_alpaca_e3_finetune.py`
```python
### PART 1中
#预训练模型存放的位置
pretrained_model_name_or_path = '/root/autodl-tmp/internlm2-1.8b-chat'
#微调数据存放的位置
data_files = '/root/public/data/target_data.json'#基座模型路径

max_length = 512
# 每一批训练样本的大小
batch_size = 2
#最大训练轮数
max_epochs = 3
#验证数据
evaluation_inputs = [
    '只剩一个心脏了还能活吗？',
    '爸爸再婚，我是不是就有了个新娘？',
    '樟脑丸是我吃过最难吃的硬糖有奇怪的味道怎么还有人买',
    '马上要上游泳课了，昨天洗的泳裤还没干，怎么办',
    '我只出生了一次，为什么每年都要庆生'
]
# PART 3中
dataset=dict(type=load_dataset, path="json",data_files=data_files)
dataset_map_fn=None
```

## 6. Execute fine-tuning
- Training
```bash
cd /root/autodl-tmp

xtuner train internlm2_1_8b_qlora_alpaca_e3.py

# Possibly missing this package in some server instance
pip install protobuf
```

- Convert checkpoint to HF format
```bash
xtuner convert pth_to_hf internlm2_1_8b_qlora_alpaca_e3.py \
  /root/autodl-tmp/work_dirs/internlm2_1_8b_qlora_alpaca_e3/iter_2000.pth \
  /root/autodl-tmp/work_dirs/internlm2_1_8b_qlora_hf
```

- Merge base model and fine-tuned LoRA model
```bash
xtuner convert merge \
/root/autodl-tmp/models/Shanghai_AI_Laboratory/internlm2-chat-1_8b \
/root/autodl-tmp/work_dirs/internlm2_1_8b_qlora_hf \
/root/autodl-tmp/work_dirs/internlm2_1_8b_qlora_merged \
--max-shard-size 2GB
```

- Chat with final model
```bash
xtuner chat \
/root/autodl-tmp/work_dirs/internlm2_1_8b_qlora_merged \
--prompt-template internlm2_chat \
--system-template colorist
```